# Track A — Single Image → Lip‑Synced Video

This Colab notebook generates a **30–45s MP4** from a **single image + TTS audio** using **Wav2Lip** and **Coqui‑TTS**.

**Pipeline**: Coqui TTS → `speech.wav` → Wav2Lip (with your image) → `final.mp4`

In [ ]:
# Cell 1 — System & deps
!nvidia-smi || true
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip -q install opencv-python moviepy pydub soundfile librosa ffmpeg-python gdown # Removed version constraint
!apt -q update && apt -q install -y ffmpeg
!pip install --upgrade pip
!pip install numpy==1.26.4 # Install a compatible version of numpy
!pip install --upgrade TTS
!pip install -q face-alignment==1.3.5
!pip install librosa==0.10.1

/bin/bash: line 1: nvidia-smi: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 69.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gruut 2.2.3 requires numpy<2.0.0,>=1.19.0, but you have numpy 2.0.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
arviz 0.22.0 requires pandas>=2.1.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatibl

In [ ]:
# 1) Clean out conflicting installs
!pip uninstall -y torch torchvision torchaudio

# 2) Reinstall all three from the SAME CUDA build (cu118 works well on Colab)
!pip install --force-reinstall --index-url https://download.pytorch.org/whl/cu118 \
  torch torchvision torchaudio

# 3) (Re)install Coqui-TTS without bumping torch/torchaudio
!pip install numpy==1.26.4 # Install a compatible version of numpy
!pip install --no-deps --upgrade TTS

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cached https://download.pytorch.org/whl/cu118/nv

  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
^C




If your **weights** (`wav2lip_gan.pth`) or **image** are in Drive, mount it here.

In [ ]:
# Cell 3 — Get Wav2Lip repo
%cd /content
!git clone https://github.com/Rudrabha/Wav2Lip.git
%cd /content/Wav2Lip
!pip -q install -r requirements.txt
%cd /content

/content
fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
/content/Wav2Lip
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86, 4.12.0.88)
ERROR: No matching distribution found fo

## Weights: wav2lip_gan.pth
- Manually upload (left pane → *files* → upload), then set the path.

In [ ]:
# Cell 4C — (Optional) If you uploaded weights manually, set path and copy
uploaded_weights = "/content/wav2lip_gan.pth"  # change if you uploaded with a different name
if os.path.exists(uploaded_weights):
    shutil.copy(uploaded_weights, "/content/Wav2Lip/wav2lip_gan.pth")
    print("Copied uploaded weights to repo.")
else:
    print("No manually uploaded weights detected at:", uploaded_weights)

## Input Image
Provide your image path. You can upload directly or use Drive. The code below shows both.

In [ ]:
# Cell 5 — Set image path
INPUT_IMAGE = "/content/image_1.jpg"  # change if you uploaded with another name
# Example test image (optional):
!wget -O /content/image_1.jpg https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg
print("Using image:", INPUT_IMAGE)

--2025-08-11 23:09:08--  https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91814 (90K) [image/jpeg]
Saving to: ‘/content/image_1.jpg’

/content/image_1.jp 100%[===================>]  89.66K  --.-KB/s    in 0.02s   

2025-08-11 23:09:08 (3.60 MB/s) - ‘/content/image_1.jpg’ saved [91814/91814]

Using image: /content/image_1.jpg


In [ ]:
from TTS.api import TTS
import torch


tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False)
device = "cuda" if torch.cuda.is_available() else "cpu"
tts.to(device)  # <-- replaces gpu=True
TEXT = "Hello! This is a demo for my Alethea AI assignment. I generated this audio and synced the lip movements to an image."
OUT_WAV = "/content/speech.wav"
tts.tts_to_file(text=TEXT, file_path=OUT_WAV)

print("Saved:", OUT_WAV)


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

### Make the audio ~30–45s (optional)
If your text is short, concatenate and trim to a target length.

In [ ]:
# Cell 7 — Pad/trim to ~35s
import subprocess
target_sec = 35
with open("/content/concat.txt", "w") as f:
    for _ in range(3):
        f.write("file '/content/speech.wav'\n")
subprocess.run(["ffmpeg","-y","-f","concat","-safe","0","-i","/content/concat.txt","-c","copy","/content/speech_long.wav"], check=True)
subprocess.run(["ffmpeg","-y","-i","/content/speech_long.wav","-t",str(target_sec),"-acodec","copy","/content/speech.wav"], check=True)
print("Prepared /content/speech.wav (~35s)")

Prepared /content/speech.wav (~35s)


In [ ]:
import re

audio_file = "/content/Wav2Lip/audio.py"

with open(audio_file, "r") as f:
    code = f.read()

patched_code = re.sub(
    r"librosa\.filters\.mel\(\s*hp\.sample_rate\s*,\s*hp\.n_fft\s*,",
    "librosa.filters.mel(sr=hp.sample_rate, n_fft=hp.n_fft,",
    code,
    count=1
)

if code != patched_code:
    with open(audio_file, "w") as f:
        f.write(patched_code)
    print("✅ Patched librosa.filters.mel() call to use keyword arguments.")
else:
    print("ℹ️ Patch not applied (line already updated or pattern not found).")


✅ Patched librosa.filters.mel() call to use keyword arguments.


In [ ]:
import re, io

path = "/content/Wav2Lip/inference.py"
with open(path, "r", encoding="utf-8") as f:
    code = f.read()

# Replace the torch.load(...) call inside _load(...) to include weights_only=False
patched = re.sub(
    r"torch\.load\((\s*checkpoint_path\s*,)",
    r"torch.load(\1 weights_only=False,",
    code,
    count=1
)

if code != patched:
    with open(path, "w", encoding="utf-8") as f:
        f.write(patched)
    print("✅ Patched inference.py to use weights_only=False")
else:
    print("ℹ️ Patch not applied (pattern not found or already patched)")


✅ Patched inference.py to use weights_only=False


In [ ]:
!curl -L \
  "https://huggingface.co/Nekochu/Wav2Lip/resolve/main/wav2lip_gan.pth" \
  -o /content/wav2lip_gan.pth
print("Downloaded wav2lip_gan.pth")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1317  100  1317    0     0   4413      0 --:--:-- --:--:-- --:--:--  4419
100  415M  100  415M    0     0  45.2M      0  0:00:09  0:00:09 --:--:-- 47.6M
Downloaded wav2lip_gan.pth


In [ ]:
# Run from repo root
%cd /content/Wav2Lip

# Ensure required folders exist
!mkdir -p temp results

# Run inference (single image)
!python inference.py \
  --checkpoint_path /content/wav2lip_gan.pth \
  --face /content/image_1.jpg \
  --audio /content/speech.wav \
  --outfile results/final.mp4 \
  --static True \
  --resize_factor 2 \
  --pads 0 10 0 0 \
  --nosmooth


/content/Wav2Lip
Using cpu for inference.
Number of frames available for inference: 1
(80, 2322)
Length of mel chunks: 722
  0% 0/6 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:02<00:00,  2.98s/it]
Load checkpoint from: /content/wav2lip_gan.pth
Model loaded
100% 6/6 [02:41<00:00, 26.86s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg 

In [ ]:
import os, shutil
print("Exists?", os.path.isfile("/content/Wav2Lip/results/final.mp4"))
shutil.copy("/content/Wav2Lip/results/final.mp4", "/content/final.mp4")
print("Saved -> /content/final.mp4")


Exists? True
Saved -> /content/final.mp4


---
### Notes / Known Issues
- TTS voice is basic; you can switch to another Coqui model name.
- Keep your input image frontal with decent lighting for best results.
- If you see CUDA OOM, reduce resolution or try a shorter audio.
